In [1]:
from jiraInteractionModules import createJIRATask, createJIRASubTask
from buildCardData import getShopifyOrderData, processShopifyOrderData, correctShopifyOrderDictionaries, buildCreateCardData
import json
import os

# Read the contents of the preferences.json file
with open(os.path.join(os.getcwd(),"preferences.json"), 'r') as file:
    preferences_data = json.load(file)

# Define JIRA credentials and project key
jira_username = preferences_data[0]["jira_username"]
jira_base_url = preferences_data[0]["jira_base_url"]
jira_project_key = preferences_data[0]["jira_project_key"]
jira_key = preferences_data[0]["jira_key"]

# Define Shopify details and credentials
shopify_key = preferences_data[0]["shopify_key"]
shopify_phrase = preferences_data[0]["shopify_phrase"]
shopify_site_name = preferences_data[0]["shopify_site_name"]
shopify_base_url = preferences_data[0]["shopify_base_url"]

shopify_request_type = "orders.json?"
#limit = 250 #currently hardset in the method
shopify_created_at_min = "2024-03-01T00:00:00-05:00"
shopify_created_at_max = "2024-03-01T00:00:00-05:00" #Currently not in use
shopify_fulfillment_status = "any" #doesn't seem to be working at this time
shopify_status = "any"
shopify_logOutputPath = "E:\\Code Projects\\ShopifyWork\\shopifyToJIRAPlug\\"

'''Start runner to get data and use it'''
data = getShopifyOrderData(shopify_key,shopify_phrase,shopify_site_name,shopify_base_url,shopify_request_type,shopify_status,shopify_created_at_min,shopify_logOutputPath)

orderDF,lineItemDF,orderNumberArray = processShopifyOrderData(data)

orderDF,lineItemDFCopy,orderNumberArray = correctShopifyOrderDictionaries(orderDF,lineItemDF,orderNumberArray)

cardInfo,subTaskCardInfo = buildCreateCardData(orderDF,lineItemDFCopy,orderNumberArray)

Order processing completed, please reference dictionaries orderDF and lineItemDF
No temp dictionary to delete, continuing process
Order number #1403 - Line item total number changed to: 2
Order number #1402 - Line item total number changed to: 1
Order number #1399 - Line item total number changed to: 4
Order number #1394 - Line item total number changed to: 1
Order number #1393 - Line item total number changed to: 1
Order number #1391 - Line item total number changed to: 4
Corrected line item dictionary created under name lineItemDFCopy
{'1406_LI_0':    order_number   line_items_id line_items_name line_items_price  \
0          1406  14765061636418     Snuggle Set            30.00   

   line_items_quantity  line_items_properties_num  \
0                    1                          6   

                         line_items_properties_names  \
0  [_has_gpo, file-1, Size, Style, Add ons, Message]   

                        line_items_properties_values  
0  [384001, https://option.nyc3

In [3]:
'''Start of method to find orders that are already fufilled and set them into their own dictionary'''
count = 0
orderCount = len(cardInfo)
holderArray = []
holderSubArray = []
subHolderArray = []
fulfilledCardDict = {}
fulfilledSubCardDict = {}

if orderCount > 0:
    while count < orderCount:
        
        if cardInfo[orderNumberArray[count]][4] == "fulfilled":
            #trigger condition for storing information to another array
            holderArray.append(str(cardInfo[orderNumberArray[count]][0]))

            key = str(orderNumberArray[count])
            fulfilledCardDict[key] = cardInfo[orderNumberArray[count]]

            #Check for line items and store in another array
            lineItemsInOrder = orderDF[orderNumberArray[count]].iloc[0]['line_items_num']
            lineCount = 0

            while lineCount < lineItemsInOrder:
                subKey = str(orderNumberArray[count]) + "_LI_" + str(lineCount)
                holderSubArray.append(subKey)
                fulfilledSubCardDict[subKey] = subTaskCardInfo[subKey]

                lineCount+=1
            
        count+=1
        
print(holderArray)
print(holderSubArray)

['1404', '1400', '1397', '1395']
['1404_LI_0', '1400_LI_0', '1397_LI_0', '1397_LI_1', '1397_LI_2', '1395_LI_0']


In [12]:
#Loop to remove fufilled cards from the original dictionary
count = 0
if len(fulfilledCardDict) > 0:
    while count < len(fulfilledCardDict):
        #Check if the desired index exists in the dictionary
        #print(str(holderArray[count]))
        if str(holderArray[count]) in cardInfo:
            #Determine line items in the given order
            lineItemsInOrder = orderDF[str(holderArray[count])].iloc[0]['line_items_num']
            lineCount = 0

            while lineCount < lineItemsInOrder:
                subKey = str(holderArray[count]) + "_LI_" + str(lineCount)
                del subTaskCardInfo[subKey]
                #print(f"Should be deleting {subKey} from subTaskCardInfo")
                lineCount+=1

            del cardInfo[str(holderArray[count])]
            #print(f"Should be deleting {holderArray[count]} from cardInfo")
        else:
            print(f"Frame for {holderArray[count]} not found in original dictionary")

        count+=1
else:
    print("No fufilled orders found in the pull")

print(cardInfo)
print(subTaskCardInfo)
print(fulfilledCardDict)
print(fulfilledSubCardDict)

Frame for 1404 not found in original dictionary
Frame for 1400 not found in original dictionary
Frame for 1397 not found in original dictionary
Frame for 1395 not found in original dictionary
{'1406': ['1406', '2024-03-26T22:38-0400', '2024-05-21', '2024-05-23', None, 'paid', '1406 - Chelsea White', '\nOrder #1406 | Placed On: 2024-03-26 22:38 EDT | Placed By: Chelsea White\nLine Item(s) Present: 1 \nLine Item Information:\n* *Name*: Snuggle Set\n** *ID*: 14765061636418\n** *Price*: 30.00\n** *Quantity*: 1\n** *Properties*: \n*** *_has_gpo*: 384001\n*** *file-1*: https://option.nyc3.digitaloceanspaces.com/uploads/77712/IMG_2831-1711506993-562062004.png\n*** *Size*: 18-24m\n*** *Style*: short sleeve/shorts\n*** *Add ons*: colorblock\n*** *Message*: ', 'https://admin.shopify.com/store/bug-bear-6049/orders/5741146898754', 1], '1405': ['1405', '2024-03-26T20:06-0400', '2024-05-21', '2024-05-23', None, 'paid', '1405 - Kayla Barnes', '\nOrder #1405 | Placed On: 2024-03-26 20:06 EDT | Placed 